In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from llm import config as config_llm
from llm.extract_text import extract
from llm.prepare_dataset import prepare_dataset
from llm.training import LLMTolkien

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/hc/93ps7hy573zg_8r1xdn4q3qm0000gn/T/tmpi_l82iky
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/hc/93ps7hy573zg_8r1xdn4q3qm0000gn/T/tmpi_l82iky/_remote_module_non_scriptable.py
/Users/andrewhuang/Desktop/llm_tolkien/venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [5]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeycha

In [8]:
extract(
    url=config_llm.url, 
    start_page=config_llm.start_page, 
    end_page=config_llm.end_page, 
    header_height=config_llm.header_height, 
    footer_height=config_llm.footer_height,
    extraction_path=config_llm.extraction_path
    )

INFO:llm.extract_text:Start extracting pages from https://gosafir.com/mag/wp-content/uploads/2019/12/Tolkien-J.-The-lord-of-the-rings-HarperCollins-ebooks-2010.pdf
INFO:llm.extract_text:Finished extracting texts from https://gosafir.com/mag/wp-content/uploads/2019/12/Tolkien-J.-The-lord-of-the-rings-HarperCollins-ebooks-2010.pdf
INFO:llm.extract_text:Start writing to /Users/andrewhuang/Desktop/llm_tolkien/llm/data/extracted_text.jsonl
0it [00:00, ?it/s]
1it [00:00,  1.64it/s]0 [00:00<?, ?it/s]
38it [00:06,  6.85it/s]0 [00:00<00:03, 369.36it/s]
54it [00:09,  3.98it/s]0 [00:06<01:40, 11.22it/s] 
63it [00:11,  5.52it/s]0 [00:09<02:06,  8.79it/s]
69it [00:12,  5.24it/s]10 [00:10<02:16,  8.08it/s]
73it [00:12,  5.79it/s]10 [00:11<02:20,  7.82it/s]
76it [00:13,  4.37it/s]10 [00:12<02:24,  7.58it/s]
78it [00:13,  5.64it/s]10 [00:12<02:35,  6.99it/s]
80it [00:14,  6.07it/s]10 [00:13<02:34,  7.05it/s]
82it [00:14,  7.16it/s]10 [00:13<02:35,  7.00it/s]
84it [00:14,  5.52it/s]10 [00:13<02:30,  7.

In [17]:
prepare_dataset(
    dataset_path=config_llm.extraction_path, 
    min_length=config_llm.min_length,
    context_length=config_llm.context_length,
    test_size=config_llm.test_size,
    shuffle=config_llm.shuffle,
    hf_repo=config_llm.hf_repo,
    model_name=config_llm.model_name,
    hf_book_repo=config_llm.hf_book_repo,
    )

INFO:llm.prepare_dataset:Start preparing dataset from /Users/andrewhuang/Desktop/llm_tolkien/llm/data/extracted_text.jsonl


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

INFO:llm.prepare_dataset:The tokenized dataset is composed of 586 elements, each one composed of 2048 tokens.
INFO:llm.prepare_dataset:The training dataset is composed of 527 elements, the test dataset is composed of 59 elements.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

INFO:llm.prepare_dataset:Preparing dataset finished.


In [2]:
lora_config = {
    "r": config_llm.lora_r,
    "lora_alpha": config_llm.lora_alpha,
    "lora_dropout": config_llm.lora_dropout, 
    'bias': config_llm.lora_bias,
    "task_type": config_llm.lora_task_type,
}

trainer_config = {
    "per_device_train_batch_size": config_llm.per_device_train_batch_size, 
    "gradient_accumulation_steps": config_llm.gradient_accumulation_steps,
    "warmup_steps": config_llm.warmup_steps,
    "weight_decay": config_llm.weight_decay,
    "num_train_epochs": config_llm.num_train_epochs,
    "learning_rate": config_llm.learning_rate, 
    "fp16": config_llm.fp16,
    "logging_steps": config_llm.logging_steps, 
    "output_dir": config_llm.hf_repo,
    "overwrite_output_dir": config_llm.overwrite_output_dir,
    "evaluation_strategy": config_llm.evaluation_strategy,
    "save_strategy": config_llm.save_strategy,
    "push_to_hub": config_llm.push_to_hub
}

model = LLMTolkien(config_llm.model_name)
model.train(
    hf_repo=config_llm.hf_repo,
    lora_config=lora_config,
    trainer_config=trainer_config,
    mlm=config_llm.mlm,
    push_to_hub=False,
)

Instantiating an MPTForCausalLM model from /Users/andrewhuang/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-storywriter/a5e85ae1941e31bb705adbcafce9b0dfd6f3a48b/modeling_mpt.py
You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        